In [8]:
!pip install pandas

In [9]:
#Importando bibliotecas
import pandas as pd
import pandas_profiling
import numpy as np
import datetime
import numpy

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Base de dados de Varejo

O sample obtido de uma empresa de marketing digital para o Varejo é um arquivo CSV de cerca de 1GB sobre 6 meses de transações
de compra de 3 lojas de um mercado tipo hortifruti. Essas compras são identificadas quanto ao consumidor no caixa no ato da compra.

Para o objetivo do trabalho iremos adotar que esse processo de identificação dos clientes no ato da compra é eficiente e que a base é representativa, ou seja, que ela consegue representar a grande maioria das compras dos clientes identificados.

A empresa de Marketing Digital deseja promover através de promoções customizadas por perfil de clientes a maior frequência de compra e o aumento do ticket médio, usando essa base e uma abordagem de aprendizado de máquina.

Por motivos legais não foi usada nenhuma informação pessoal das pessoas, portanto a abordagem usará fundamentalmente o que esses consumidores estão comprando, com que frequência, etc.

Sabemos desta base que precisamos o ID da transação para identificar os itens de uma mesma cesta, o ID dos clientes para identificar as compras por cliente, o ID e descrição dos produtos comprados, o valor e quantidade de produtos comprados.

Portanto a primeira etapa será criar uma base com essas dimensões.

In [10]:
# Lendo os dados
df = pd.read_csv('C:\\users\\tabuq\\Projeto Final\\dataset\\dados_cliente_1.csv', delimiter=',', 
                 usecols = ['cod_transacao', 'cod_pessoa', 'dat_compra', 'cod_loja', 'cod_produto', 'des_produto', 
                        'vlr_item_compra', 'qtd_item_compra'], parse_dates=['dat_compra'], 
                 dtype={'cod_transacao': int, 'cod_pessoa': int, 'cod_loja': int, 'cod_produto': int, 
                        'des_produto': str, 'vlr_item_compra' : float, 'qtd_item_compra' : float})

In [11]:
df.shape
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9649885 entries, 0 to 9649884
Data columns (total 8 columns):
 #   Column           Dtype         
---  ------           -----         
 0   cod_transacao    int32         
 1   cod_pessoa       int32         
 2   dat_compra       datetime64[ns]
 3   cod_loja         int32         
 4   des_produto      object        
 5   cod_produto      int32         
 6   vlr_item_compra  float64       
 7   qtd_item_compra  float64       
dtypes: datetime64[ns](1), float64(2), int32(4), object(1)
memory usage: 441.7+ MB


In [12]:
df.head (10)

,cod_transacao,cod_pessoa,dat_compra,cod_loja,des_produto,cod_produto,vlr_item_compra,qtd_item_compra
0,6504312,11498,2020-01-02 07:11:03,1,CAR SUI LOMBO KG,465436288,19.49,1.000
1,6504312,11498,2020-01-02 07:11:03,1,MORT PERD BOLOGNELA KG,-1359334784,16.91,0.304
2,6504312,11498,2020-01-02 07:11:03,1,CAR BOV COXAO MOLE KG,1147333120,27.98,1.088
3,6504312,11498,2020-01-02 07:11:03,1,PAO FRANCES M BEM+ KG,-440312192,9.89,0.470
4,6504312,11498,2020-01-02 07:11:03,1,CAR AVE COXINHA ASA KG,1037206016,14.98,1.016
5,6504312,11498,2020-01-02 07:11:03,1,BATATA COOLMIND CONG TRA,929272506,19.98,1.000
6,6504313,8935,2020-01-02 07:12:36,1,FRUTAS MANGA TOMMY ATKIN,-104694528,1.98,0.810
7,6504313,8935,2020-01-02 07:12:36,1,FRUTAS PESSEGO NAC KG,431123072,5.98,0.560
8,6504313,8935,2020-01-02 07:12:36,1,PAO FRANCES M BEM+ KG,-440312192,9.91,0.115
9,6504313,8935,2020-01-02 07:12:36,1,BISC TUC S TRADICIONAL 1,1911410629,1.29,1.000


In [13]:
df.tail(10)

,cod_transacao,cod_pessoa,dat_compra,cod_loja,des_produto,cod_produto,vlr_item_compra,qtd_item_compra
9649875,7152988,83195,2020-06-30 21:17:17,2,REFR PO TRINK LARANJA 25,1861236971,0.68,2.0
9649876,7152988,83195,2020-06-30 21:17:17,2,REFR PO TRINK UVA 25G,1861237023,0.68,2.0
9649877,7152988,83195,2020-06-30 21:17:17,2,PAP HIG DUETTO FIT NEUT,1876910552,11.99,1.0
9649878,7152988,83195,2020-06-30 21:17:17,2,SAL GROSSO CISNE CHURRAS,1885319970,2.98,1.0
9649879,7152988,83195,2020-06-30 21:17:17,2,ACHOC PO SIAMAR SOLUVEL,1919638352,3.59,1.0
9649880,7152988,83195,2020-06-30 21:17:17,2,"GUARDANAPO MILI 30X29,50",1955104044,3.09,1.0
9649881,7152988,83195,2020-06-30 21:17:17,2,LEITE LIDER INT 1L,-1875452341,3.59,12.0
9649882,7152988,83195,2020-06-30 21:17:17,2,CAFE SABORIAL EXTFORT 5,-1633657302,5.99,1.0
9649883,7152988,83195,2020-06-30 21:17:17,2,ACUCAR SANTA ISABEL CRIS,-257027323,4.19,1.0
9649884,7152988,83195,2020-06-30 21:17:17,2,RACAO BEAUTYPET CARNE 6K,861168913,27.90,1.0


In [14]:
# Avaliando os dados quanto a presença de null
df.isnull().values.any()
df.isnull().values.sum()
df.isnull().sum()*100/df.shape[0]

cod_transacao      0.0
cod_pessoa         0.0
dat_compra         0.0
cod_loja           0.0
des_produto        0.0
cod_produto        0.0
vlr_item_compra    0.0
qtd_item_compra    0.0
dtype: float64

Pode ser observado que a base possui cerca de 9,6 milhões de registros, relativos ao período de janeiro a junho de 2020.
Não há registros nulos em nenhuma dimensão. O unico ponto observado seria que o código de produto apresenta valores inteiros
negativos, o que não seria esperado para esses códigos (cod_produto).

In [15]:
# Convertendo cod_produto para valores positivos
df['cod_produto'] = df['cod_produto'].abs()

In [16]:
df.head(10)

,cod_transacao,cod_pessoa,dat_compra,cod_loja,des_produto,cod_produto,vlr_item_compra,qtd_item_compra
0,6504312,11498,2020-01-02 07:11:03,1,CAR SUI LOMBO KG,465436288,19.49,1.000
1,6504312,11498,2020-01-02 07:11:03,1,MORT PERD BOLOGNELA KG,1359334784,16.91,0.304
2,6504312,11498,2020-01-02 07:11:03,1,CAR BOV COXAO MOLE KG,1147333120,27.98,1.088
3,6504312,11498,2020-01-02 07:11:03,1,PAO FRANCES M BEM+ KG,440312192,9.89,0.470
4,6504312,11498,2020-01-02 07:11:03,1,CAR AVE COXINHA ASA KG,1037206016,14.98,1.016
5,6504312,11498,2020-01-02 07:11:03,1,BATATA COOLMIND CONG TRA,929272506,19.98,1.000
6,6504313,8935,2020-01-02 07:12:36,1,FRUTAS MANGA TOMMY ATKIN,104694528,1.98,0.810
7,6504313,8935,2020-01-02 07:12:36,1,FRUTAS PESSEGO NAC KG,431123072,5.98,0.560
8,6504313,8935,2020-01-02 07:12:36,1,PAO FRANCES M BEM+ KG,440312192,9.91,0.115
9,6504313,8935,2020-01-02 07:12:36,1,BISC TUC S TRADICIONAL 1,1911410629,1.29,1.000


In [17]:
# Exportanto o dataframe para um novo arquivo
df.to_csv('C:\\users\\tabuq\\Projeto Final\\dataset\\base_varejo.csv', index=False)